# Multi-Class Classification - Language Classification

This notebook implements the method presented in Goldberg's [2017] book "Neural Network Methods for Natural Language Processing". It shows the steps you need to go through in order to successfully train a classifier, and it should also, so I hope, illustrate the notational differences between Goldberg and standard machine learning literature.

$NOTE$: There is no cross-validation etc. to find optimal parameters. This is simply to show how multi-class classification works. This will be part of a tutorial session and all other concepts will be explained there.

Author: Phillip Ströbel

## Getting and cleaning the data

The data consists of downloaded Wikipedia articles (see `urls.txt`) in German, English, French, Spanish, Italian and Finnish (instead of "O" in Goldberg). The data is in HTML, so we need to some preprocessing to get the text out of it. We also restrict ourselfes to the characters from a to z in the alphabet (as described in Goldberg). In this fashion, we get rid of all the Umlauts (ä, ö, ü) and all other characters with diacritics (as, e.g., the é or ç in French). Note however, that if these characters ocurring in bigrams would probably be good features. In some way, we still keep the information "special character" by not fully deleting the character, but by replacing it by the dollar sign "\$". Furthermore, we replace all punctuation marks and digits by dollar signs as well. As such, all special characters, digits, and punctuation marks are mapped to $. The space will be replaced by an underscore "\_". We then represent each langauge by 28 characters, as is suggested by Goldberg.

### Cleaning HTML
We first strip the HTML to get only the text of the Wikipedia page.

#### Get the html files

In [4]:
import re
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
from collections import defaultdict

seed = np.random.seed(seed=200)  # set a seed for random, so results are reproducible

article_dict = defaultdict(lambda: defaultdict(str))

regex = r'[\n ]{2,}'
pattern = re.compile(regex)

urls = open('urls.txt', 'r').readlines()

for index, url in enumerate(urls):
    language = url[8:10]
    doc_id = 'doc_%d' % index
    html = urlopen(url.strip()).read()    
    soup = BeautifulSoup(html, 'html.parser')
    raw = soup.body.get_text()  # only get text from the text body (this excludes headers and should exclude navigation bars)
    raw = re.sub(pattern, ' ', raw)  # replace multiple breaks and spaces by only one space
    raw = re.sub(r'\n', ' ', raw)  # replace every line break with a space
    article_dict[language][doc_id] = raw.lower()  # assign each text to its language and lower all uppercase characters

### Preprocessing --> prepare the text
replace special characters and digits

In [5]:
preprocessed_dict = defaultdict(lambda: defaultdict(str))

abc = r'[a-z]'
abc_pattern = re.compile(abc)

for lang, doc in article_dict.items():
    for doc, text in doc.items():
        for char in text:
            if re.match(abc_pattern, char):
                preprocessed_dict[lang][doc] += char
            elif re.match(' ', char):
                preprocessed_dict[lang][doc] += '_'
            else:
                preprocessed_dict[lang][doc] += '$'

### Count bigrams --> Feature extraction

The distribution of bigrams will be our only feature. We could extend this by taking into account other n-grams.

In [6]:
charset = 'abcdefghijklmnopqrstuvwxyz$_'  # define the character set we want to use

In [7]:
from itertools import combinations_with_replacement, permutations

def bigrams(text):
    """
    Function to extract bigrams from text and calculate their distribution
    :param text: text string
    :return: dictionary containing bigrams as keys, and the normalised count as values
    """
    combs = combinations_with_replacement(charset, 2)
    perms = permutations(charset, 2)
    bigram_dict = dict()
    
    for comb in set(list(combs) + list(perms)):
        bigram_dict[''.join(comb)] = 0
        
    doc_length = len(text)
    
    for index in range(0, len(text)-1):
        bigram = text[index] + text[index+1]
        bigram_dict[bigram] += 1
                
    for bigram, count in bigram_dict.items():
        bigram_dict[bigram] = count/doc_length

    return bigram_dict              

### Put data into pandas dataframe
The pandas dataframe allows us to conveniently represent all the data we need in one table. So let's do this. But first we need to extract the features.

In [8]:
bigram_dict_full = defaultdict(lambda: defaultdict(dict))

for lang, doc in preprocessed_dict.items():
    for doc, text in sorted(doc.items()):
        bigram_dict = bigrams(text)
        bigram_dict_full[lang][doc] = bigram_dict

In [9]:
import pandas as pd

col_names = ['y'] + sorted(bigram_dict_full['en']['doc_0'].keys())
my_df = dict()

for col in col_names:
    my_df[col] = list()
    
df = pd.DataFrame(my_df)

for lang, doc in bigram_dict_full.items():
    for key, value in doc.items():
        df_obj = value
        df_obj['y'] = lang
        df = df.append(df_obj, ignore_index=True)
        
df.head()
        

,$$,$_,$a,$b,$c,$d,$e,$f,$g,$h,...,zq,zr,zs,zt,zu,zv,zw,zx,zy,zz
0,0.100732,0.036889,0.000292,0.000222,0.000332,0.000163,0.000239,0.000181,0.000099,0.000152,...,0.000000,0.000006,0.000006,0.0,0.000023,0.000000,0.000000,0.0,0.000000,0.000006
1,0.062187,0.028911,0.000513,0.000322,0.000513,0.000356,0.000629,0.000192,0.000185,0.000171,...,0.000007,0.000007,0.000000,0.0,0.000027,0.000000,0.000000,0.0,0.000014,0.000021
2,0.072008,0.031248,0.000444,0.000519,0.000497,0.000187,0.000340,0.000246,0.000093,0.000179,...,0.000000,0.000004,0.000000,0.0,0.000030,0.000004,0.000004,0.0,0.000011,0.000000
3,0.075458,0.035249,0.000400,0.000466,0.000453,0.000268,0.000303,0.000224,0.000312,0.000189,...,0.000000,0.000004,0.000000,0.0,0.000013,0.000000,0.000000,0.0,0.000009,0.000000
4,0.064235,0.032112,0.000344,0.000301,0.000735,0.000295,0.000476,0.000313,0.000271,0.000187,...,0.000000,0.000000,0.000006,0.0,0.000006,0.000000,0.000000,0.0,0.000000,0.000000


In [10]:
df.shape

(60, 785)

Now we split the data into the label vector \begin{equation}\mathbf{y}\end{equation} and a training data matrix \begin{equation}\mathbf{X}\end{equation}. But first, we shuffle the df and split it into a training and a test set.

Moreover, it is necessary for many machine learning tasks to standardise the data. Our aim is for each feature to be represented by a column vector in which values have zero mean and unit variance.

In [11]:
def normalise_point(datapoint, mean, std):
    """
    normalise a datapoint to zero mean and unit variance.
    :param datapoint: value as a float
    :param mean: mean of data vector x
    :param std: standard deviation of data vector x
    :return: normalised datapoint (float)
    """
    return (datapoint - mean)/std

def normalise_matrix(matrix):
    """
    normalises the data matrix
    :param matrix: input matrix
    :return: normalised matrix
    """
    train_normalised = matrix.copy()
    
    for col in matrix:
        try:
            mean = matrix[col].mean()
            std = matrix[col].std()
            for index, item in enumerate(matrix[col]):
                train_normalised.loc[index, col] = normalise_point(item, mean, std)
        except ZeroDivisionError:
            train_normalised.loc[index, col] = 0.0
        except TypeError:
            continue
    return train_normalised

In [12]:
df_norm = normalise_matrix(df)

Split the data into a train set and a test set

In [13]:
train = df_norm.sample(frac=0.9, random_state=seed)
test = df_norm.drop(train.index)

Define the different sets

In [14]:
# for training
y_train = train.y
X_train = train.drop('y', axis=1)

# for testing
y_test = test.y
X_test = test.drop('y', axis=1)

Check the shapes

In [15]:
print('Training samples shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test samples shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training samples shape:  (54, 784)
Training labels shape:  (54,)
Test samples shape:  (6, 784)
Test labels shape:  (6,)


We should binarise our labels, although libraries like sklearn can also deal with non-numeric data

In [16]:
from sklearn import preprocessing

lb = preprocessing.LabelBinarizer()
lb.fit(['en', 'fr', 'de', 'it', 'es', 'fi'])

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [17]:
lb.classes_

array(['de', 'en', 'es', 'fi', 'fr', 'it'], dtype='<U2')

We do this for both our training and test labels:

In [18]:
y_train = lb.transform(y_train)
y_test = lb.transform(y_test)

Labels are now one-hot encoded:

In [19]:
y_train[0]

array([0, 0, 0, 1, 0, 0])

We almost have everything now. However, we need to take care of the bias and the weight matrix. The hypothesis ŷ is given by:
\begin{equation}
\mathbf{\hat{y}}=\mathbf{x}\cdot\mathbf{W}+\mathbf{b}
\end{equation}
We can achieve this by appending 1 to each feature vector x, and the whole weight vector b to the weight matrix W. This is called the bias trick. Note that the dimensions of X_train change, and that the weight matrix W will have match the dimensions (same number of rows as X has columns).

In [20]:
bias_vector = np.ones([X_train.shape[0], 1])
X_train['bias'] = bias_vector
X_train

,$$,$_,$a,$b,$c,$d,$e,$f,$g,$h,...,zr,zs,zt,zu,zv,zw,zx,zy,zz,bias
17,0.416581,0.724355,0.415723,-0.678734,-0.973332,-0.650595,-0.396620,-0.943649,-0.864329,0.840628,...,-0.543066,-0.498601,-0.428053,-0.440921,-0.281797,-0.418054,-0.275911,-0.448893,-0.452825,1.0
5,1.120851,0.754248,-0.134649,-0.331625,-0.658614,-0.085545,-0.823966,-0.596442,-0.633240,-0.321978,...,-0.543066,-0.498601,-0.335196,-0.443902,-0.281797,-0.418054,-0.275911,0.247155,-0.427383,1.0
33,-0.325331,-0.585072,-1.236515,1.877263,-0.211181,-0.116645,-0.431707,-0.030080,0.331097,0.853819,...,0.710842,1.221637,2.202934,2.650994,-0.281797,2.054151,-0.275911,0.646756,-0.332681,1.0
48,-1.079756,0.965223,0.604572,-0.770761,-0.413110,-0.474208,-0.162656,-0.937676,-0.877084,-1.127069,...,-0.543066,-0.498601,-0.428053,-0.453571,-0.281797,-0.418054,-0.275911,-0.448893,1.422711,1.0
58,-1.162418,-1.480115,1.662141,-0.692058,0.422829,-0.342705,-0.757610,4.230831,1.265686,-1.002417,...,-0.543066,-0.498601,-0.428053,-0.412307,-0.281797,-0.418054,-0.275911,-0.448893,-0.452825,1.0
2,0.479922,-0.069024,-1.016872,-0.317584,-0.651994,-0.889686,-0.865250,-0.604430,-0.915983,-0.875618,...,-0.089210,-0.498601,-0.428053,-0.433828,0.414915,-0.404073,-0.275911,-0.052319,-0.452825,1.0
36,-0.419403,-0.629671,-0.245953,0.937869,-0.465707,0.939226,-0.617174,0.122534,0.615506,1.581265,...,2.391855,0.508005,1.453602,2.812156,1.970898,1.932708,-0.275911,1.260772,-0.405957,1.0
54,0.084788,-0.117571,0.370100,-0.561941,0.821552,-0.558541,0.326848,-0.428954,0.236760,-0.813851,...,-0.543066,0.051543,-0.428053,-0.450279,-0.281797,-0.418054,-0.275911,-0.448893,-0.375979,1.0
46,-0.870141,1.050332,0.771331,-0.120226,-0.119737,0.866913,-0.414991,-0.356353,-0.508410,-0.345873,...,0.986294,-0.498601,-0.428053,-0.438472,-0.281797,-0.418054,-0.275911,-0.003447,1.256774,1.0
51,-0.585015,-0.009095,-0.342072,-0.744674,-0.604092,-0.743825,-0.442110,-0.326296,-0.867174,-1.005133,...,-0.543066,-0.498601,-0.428053,-0.463298,-0.281797,-0.418054,2.072622,-0.448893,4.015987,1.0


In [21]:
# initialise weight matrix with small weights

np.random.seed(seed=200)

W = np.random.randn(X_train.shape[1], len(lb.classes_)) * 0.0001
#W = np.zeros([X.shape[1], len(lb.classes_)])

We see that the dimensions are right. The dot product of a specific row from X_train and the weight matrix W constitutes a forward pass and calculates the score for each class.

In [22]:
W.shape

(785, 6)

In [23]:
X_train.shape

(54, 785)

In [24]:
X_train[5:6].dot(W)

,0,1,2,3,4,5
2,-0.002751,0.001102,0.000353,-0.000783,0.003843,-0.000011


We see that the values for the highest score of the dot product is not the score of the true label. Our aim is to change this by implementing a support vector classifier.

In [25]:
X_train[5:6].dot(W).max(axis=1)

2    0.003843
dtype: float64

In [26]:
X_train[5:6].dot(W)[y_train[5:6].argmax()]

2    0.001102
Name: 1, dtype: float64

Important: we follow kind of a naive implementation. The aim is to be able to understand what is going on!

In order to quantify how good (or how bad) our weight matrix W can predict the data in our training set, we need to implement a loss function. Here we take a go at the hinge loss, which tries to predict the correct class with a margin of at least one to all other classes (or in this case, like presented in Goldberg, to the class which does not equal the true class, but which scores highest). In my understanding, this is a one-vs-one approach (true class vs. class with highest score (but doesn't equal the true class)).

In [27]:
def multiclass_hinge_loss(x, y, W, index):
    """
    Calculates the loss of a single data point by taking the prediction of the correct value and the the prediction of
    the value of next highest score, following Crammer and Singer (2001)
    :param x: sample point x as a vector
    :param y: correct label y for x as a vector
    :param W: weight matrix
    :param index: column index of data matrix X
    :return: loss
    """
    loss = 0
    y_index = y[index].argmax()
    y_value = x.dot(W)[y_index]
    y_hat_max_value = np.delete(x.dot(W), y_index).max()
    #for j in range(0, y.shape[1]):  # in case we wanted to classify against all other classes (one-vs-all) --> currently one-vs-one
        #if j == y_index:
            #continue
    loss += max(0, 1 - (y_value - y_hat_max_value))
    return loss

With matrix multiplication, we could get all the scores at once. In the following, however, we focus on an approach which takes sample by sample and calculates the loss and the gradients.

In [28]:
scores = X_train.dot(W)  # simple matrix multiplication to get all scores

In [29]:
scores

,0,1,2,3,4,5
17,-0.007375,-0.005238,-0.004242,-0.006294,0.000810,0.003542
5,-0.005797,-0.006754,0.001188,-0.000472,0.000874,-0.001011
33,0.003550,0.001359,-0.001113,-0.000362,-0.000552,0.003478
48,0.002508,0.002688,0.002068,0.000299,-0.000161,-0.002079
58,0.000666,-0.000814,0.006181,0.002919,-0.002478,0.000340
2,-0.002751,0.001102,0.000353,-0.000783,0.003843,-0.000011
36,0.000469,-0.005094,-0.002870,0.004014,0.001441,-0.000630
54,-0.001155,0.004902,0.003001,0.005857,-0.001278,0.004642
46,-0.002020,0.000084,0.001625,-0.000914,-0.001955,-0.004747
51,-0.000868,0.004471,0.001188,0.001433,0.001760,-0.004693


In [40]:
def gradient(X, y, W):
    """
    compute the gradient
    :param X: data matrix (train) 
    :param y: the corresponding 
    :param W: weight matrix
    :return: loss and Jacobian dW with all gradients
    """
    dW = np.zeros(W.shape)
    
    total_loss = 0.0
    
    for index, x in enumerate(X.as_matrix()):
        y_index = y[index].argmax()
        y_value = x.dot(W)[y_index]
        y_hat_max_value = np.delete(x.dot(W), y_index).max()
        loss = max(0, 1 - (y_value - y_hat_max_value))
        total_loss += loss
        y_hat_max_index = np.delete(x.dot(W), y_index).argmax() + 1
        dW[:, y_hat_max_index] += x.transpose()
        dW[:, y_index] -= x.transpose()
            
    return total_loss, dW
    

In [43]:
def gradient_descent(X, y, W, eta, steps):
    """
    Perform gradient descent for a number of times with a fixed learning rate eta
    :param X: data matrix
    :param y: labels
    :param W: weight matrix
    :param eta: learning rate
    :param steps: number of times gradient descent should be performed
    :return: learned representation matrix W_learned
    """
    W_learned = W.copy()
    
    for step in range(0, steps):
        loss, dW = gradient(X, y, W_learned)
        W_learned = W_learned - eta * dW
        print(loss)
        
    return W_learned
    

In [44]:
W_star = gradient_descent(X_train, y_train, W, eta=0.001, steps=1000)

54.20622198371133
21.490525959054526
0.07620527271920619
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

### Testing
Let's test if our learned representation of the data is any good at classifying the data in the test set. Of course we need the bias in our test set as well!

In [45]:
bias_vector_test = np.ones([X_test.shape[0], 1])
X_test['bias'] = bias_vector_test

In [46]:
for index, x in enumerate(X_test.dot(W_star).as_matrix()):
    pred = x.argmax()
    true_label = y_test[index].argmax()
    print(pred, true_label)

1 1
3 3
3 3
4 4
0 0
5 5


Not too bad! But Goldberg mentioned something about regularisation, so we should take this into account as well!

In [35]:
def gradient_reg(X, y, W, lam):
    """
    compute the gradient
    :param X: data matrix (train) 
    :param y: the corresponding 
    :param W: weight matrix
    :param lam: reguliser lambda
    :return: Jacobian dW with all gradients
    """
    dW = np.zeros(W.shape)
    
    for index, x in enumerate(X.as_matrix()):
        y_index = y[index].argmax()
        y_value = x.dot(W)[y_index]
        y_hat_max_value = np.delete(x.dot(W), y_index).max()
        loss = max(0, 1 - (y_value - y_hat_max_value)) + lam * np.linalg.norm(W, 2)
        y_hat_max_index = np.delete(x.dot(W), y_index).argmax() + 1
        dW[:, y_hat_max_index] += x.transpose()
        dW[:, y_index] -= x.transpose()
        
    dW += 2 * lam * W
            
    return dW

def gradient_descent_reg(X, y, W, eta, steps):
    """
    Perform gradient descent for a number of times with a fixed learning rate eta
    :param X: data matrix
    :param y: labels
    :param W: weight matrix
    :param eta: learning rate
    :param steps: number of times gradient descent should be performed
    :return: learned representation matrix W_learned
    """
    W_learned = W.copy()
    
    for step in range(0, steps):
        dW = gradient_reg(X, y, W, 10000)
        W_learned = W_learned - eta * dW
        
    return W_learned
    

In [36]:
W_star_reg = gradient_descent_reg(X_train, y_train, W, eta=0.001, steps=1000)

In [37]:
for index, x in enumerate(X_test.dot(W_star_reg).as_matrix()):
    pred = x.argmax()
    true_label = y_test[index].argmax()
    print(pred, true_label)

1 1
3 3
3 3
5 4
0 0
5 5


If we look at the two different weight matrices (one regularised, the other not), we notice the following:

In [38]:
W_star[0:5]

array([[ -3.62888535,   6.04994232,  -2.86310234,  10.24142028,
         -5.07025647,  -4.72899317],
       [ -8.29128073,   3.76352834,  -8.44622223,   2.74631596,
          5.51610756,   4.71172435],
       [ -7.4413166 ,  -6.07465375,   7.14779523,  -9.16551201,
          5.69735729,   9.83666962],
       [ 15.87176373,  -3.16702729,   0.82967888,  -6.72842564,
          3.885268  , -10.69136123],
       [ -0.65747427,  -4.1231676 ,   2.66509709,  -9.73523486,
          8.2896523 ,   3.56119809]])

In [39]:
W_star_reg[0:5]

array([[-7.26988850e-01,  2.22803605e+00, -4.28686065e+00,
         1.07368969e+01, -5.79318894e+00, -4.66309383e+00],
       [-7.84858729e+00,  2.80901479e+00, -7.06234349e+00,
         1.16230410e+00,  5.36960929e+00,  2.10515229e+00],
       [-7.86827958e+00, -8.10935165e+00,  3.32437167e+00,
        -8.10616875e+00,  2.01308697e+00,  1.19511398e+01],
       [ 1.75975963e+01, -3.64229029e+00,  3.13804242e+00,
        -9.15839372e+00,  6.47278616e+00, -1.23368068e+01],
       [ 6.82798763e-03, -1.56030958e+00,  2.22802146e+00,
        -1.39021833e+01,  8.78608843e+00,  3.02627729e+00]])

## By the way ...
### In scikit-learn it's much easier to implement this :-)


In [39]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0, multi_class='crammer_singer', loss='hinge')
clf.fit(X_train, train.y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000,
     multi_class='crammer_singer', penalty='l2', random_state=0,
     tol=0.0001, verbose=0)

In [40]:
clf.predict(X_test)

array(['en', 'fi', 'fi', 'fr', 'de', 'it'], dtype=object)

In [41]:
test.y

4     en
12    fi
16    fi
26    fr
41    de
42    it
Name: y, dtype: object

We see that with our naive implementation, we do not much worse than with scikit's. scikit's implementation is of course much more elaborate and uses the vectorised operation and possibly other optimisation techniques in order to make its SVM (or SVC) better.